In [1]:
import numpy as np
from scipy.stats import unitary_group
import os

# --- 1. Configuration ---
DATASET_SIZE = 10000  # Number of examples to generate
N_SHOTS = 1024        # Shots per basis (Z, X, Y)
SAVE_DIR = "qst_data_ml"

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# --- 2. Helper: Generate Random Density Matrices ---
def generate_random_rho(dim=2):
    """
    Generates a random valid density matrix using the Ginibre ensemble.
    This covers both pure and mixed states.
    """
    # A is a random complex matrix
    A = np.random.randn(dim, dim) + 1j * np.random.randn(dim, dim)
    # rho = A * A_dagger
    rho = A @ A.conj().T
    # Normalize trace to 1
    rho /= np.trace(rho)
    return rho

# --- 3. Helper: Measurement Operators (From Assignment 1) ---
# We define the 6 projectors: Z0, Z1, X0, X1, Y0, Y1
s0 = np.eye(2)
sz = np.array([[1, 0], [0, -1]])
sx = np.array([[0, 1], [1, 0]])
sy = np.array([[0, -1j], [1j, 0]])

# Projectors P = (I +/- sigma) / 2
projectors = {
    'Z0': (s0 + sz)/2, 'Z1': (s0 - sz)/2,
    'X0': (s0 + sx)/2, 'X1': (s0 - sx)/2,
    'Y0': (s0 + sy)/2, 'Y1': (s0 - sy)/2
}
proj_list = [projectors[k] for k in ['Z0', 'Z1', 'X0', 'X1', 'Y0', 'Y1']]

# --- 4. Main Data Generation Loop ---
print(f"Generating {DATASET_SIZE} samples...")

inputs = []  # To store measurement frequencies (the "features")
labels = []  # To store the flattened density matrix (the "truth")

for i in range(DATASET_SIZE):
    # A. Create a random state
    rho = generate_random_rho()
    
    # B. Simulate Measurement (Born Rule + Noise)
    freqs = []
    for P in proj_list:
        # Probability p = Tr(rho * P)
        prob = np.real(np.trace(rho @ P))
        # Ensure numerical stability
        prob = np.clip(prob, 0, 1)
        
        # Simulate experimental noise (Binomial distribution)
        # Counts = np.random.binomial(n=N_SHOTS, p=prob)
        # We store the *Frequency* (Counts / Total Shots)
        simulated_counts = np.random.binomial(N_SHOTS, prob)
        freqs.append(simulated_counts / N_SHOTS)
    
    # C. Store Data
    inputs.append(freqs)
    # Store rho as a 4x1 vector of complex numbers, or separated real/imag
    # For simplicity, we keep it as a 2x2 matrix in the numpy array
    labels.append(rho)

# --- 5. Save to Disk ---
inputs = np.array(inputs, dtype=np.float32)  # Shape: (10000, 6)
labels = np.array(labels, dtype=np.complex64) # Shape: (10000, 2, 2)

save_path = os.path.join(SAVE_DIR, "dataset.npz")
np.savez(save_path, X=inputs, y=labels)

print(f"✅ Data Generation Complete!")
print(f"   Inputs Shape: {inputs.shape} (6 probabilities per sample)")
print(f"   Labels Shape: {labels.shape} (2x2 density matrix per sample)")
print(f"   Saved to: {save_path}")

Generating 10000 samples...
✅ Data Generation Complete!
   Inputs Shape: (10000, 6) (6 probabilities per sample)
   Labels Shape: (10000, 2, 2) (2x2 density matrix per sample)
   Saved to: qst_data_ml\dataset.npz


In [2]:
import torch
import torch.nn as nn

# --- 1. The Physics-Informed Neural Network ---
class QuantumStateReconstructor(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=64):
        super().__init__()
        
        # A simple 3-layer MLP (Hardware-friendly)
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            # Output layer: We need 4 real numbers to define a 2x2 L matrix
            # (2 Real Diagonals + 1 Complex Off-Diagonal = 4 params)
            nn.Linear(hidden_dim, 4) 
        )

    def forward(self, x):
        # x shape: (batch_size, 6)
        
        # 1. Get raw outputs from the neural net
        logits = self.network(x)
        
        # 2. Reshape into L matrix components
        # We need L = [[l00, 0], [l10, l11]]
        # l00, l11 are real. l10 is complex (real + imag).
        
        l00 = logits[:, 0]
        l11 = logits[:, 1]
        l10_real = logits[:, 2]
        l10_imag = logits[:, 3]
        
        # 3. Construct the Lower Triangular Matrix L
        batch_size = x.shape[0]
        L = torch.zeros((batch_size, 2, 2), dtype=torch.complex64, device=x.device)
        
        # Fill the matrix elements
        L[:, 0, 0] = l00.type(torch.complex64)
        L[:, 1, 0] = torch.complex(l10_real, l10_imag)
        L[:, 1, 1] = l11.type(torch.complex64)
        
        # 4. Compute rho = L @ L_dagger
        # Batch Matrix Multiplication (bmm)
        L_dagger = torch.transpose(L.conj(), 1, 2)
        rho_unscaled = torch.bmm(L, L_dagger)
        
        # 5. Normalize (Trace = 1)
        # Trace is the sum of diagonal elements (dim1=1, dim2=2)
        trace = torch.diagonal(rho_unscaled, dim1=1, dim2=2).sum(dim=1)
        
        # Add epsilon to prevent divide-by-zero
        trace = trace.view(-1, 1, 1) + 1e-8
        
        rho = rho_unscaled / trace
        
        return rho

# --- 2. Verification Test ---
# Let's verify the model outputs valid quantum states immediately.
if __name__ == "__main__":
    model = QuantumStateReconstructor()
    
    # Fake input: 5 examples of measurement data
    dummy_input = torch.rand(5, 6) 
    
    # Predict
    predicted_rho = model(dummy_input)
    
    print("Model Architecture Created Successfully!")
    print(f"Output Shape: {predicted_rho.shape} (Should be 5, 2, 2)")
    
    # Check Trace Constraint
    traces = torch.diagonal(predicted_rho, dim1=1, dim2=2).sum(dim=1)
    print(f"Traces (should be all ~1.0): {traces.real.detach().numpy()}")
    
    # Check if Hermitian (rho == rho_dagger)
    rho_dagger = torch.transpose(predicted_rho.conj(), 1, 2)
    is_hermitian = torch.allclose(predicted_rho, rho_dagger, atol=1e-6)
    print(f"Is Hermitian? {is_hermitian}")

Model Architecture Created Successfully!
Output Shape: torch.Size([5, 2, 2]) (Should be 5, 2, 2)
Traces (should be all ~1.0): [0.99999917 0.9999995  0.9999994  0.9999996  0.9999987 ]
Is Hermitian? True


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os

# --- 1. Load Data ---
print("Loading dataset...")
data = np.load("qst_data_ml/dataset.npz")
X_numpy = data['X']  # Shape: (10000, 6)
y_numpy = data['y']  # Shape: (10000, 2, 2)

# Convert to PyTorch Tensors
# Inputs are Float32
X_tensor = torch.tensor(X_numpy, dtype=torch.float32)
# Labels are Complex64
y_tensor = torch.tensor(y_numpy, dtype=torch.complex64)

# Split into Train (80%) and Test (20%)
split_idx = int(0.8 * len(X_tensor))
X_train, X_test = X_tensor[:split_idx], X_tensor[split_idx:]
y_train, y_test = y_tensor[:split_idx], y_tensor[split_idx:]

print(f"Data Loaded. Training on {len(X_train)} samples, Testing on {len(X_test)}.")

# --- 2. Setup Training ---
# Re-initialize the model
model = QuantumStateReconstructor() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loss Function: Mean Squared Error between the two density matrices
# We calculate Norm(rho_pred - rho_true)^2
def complex_mse_loss(pred, target):
    diff = pred - target
    # Frobenius norm squared
    loss = torch.norm(diff) ** 2
    return loss / len(pred) # Normalize by batch size

# Helper for Fidelity (Metric)
def calculate_batch_fidelity(pred, target):
    # For 2x2, simpler fidelity formula or just use overlap for pure states
    # But since these are mixed, we compute standard Fidelity approximation or exact
    # F(rho, sigma) = (Tr(sqrt(sqrt(rho) sigma sqrt(rho))))^2
    # For training speed, we'll assume pure states overlap approx: Tr(rho * sigma)
    # (Note: This is exact for pure states, approximation for mixed)
    
    # Let's use a simpler proxy for monitoring: Real part of Trace(pred * target)
    overlap = torch.matmul(pred, target)
    fid = torch.diagonal(overlap, dim1=1, dim2=2).sum(dim=1).real
    return fid.mean().item()

# --- 3. Training Loop ---
EPOCHS = 50
BATCH_SIZE = 32

print(f"\nStarting Training for {EPOCHS} epochs...")

for epoch in range(EPOCHS):
    model.train()
    
    # Mini-batch shuffling
    indices = torch.randperm(len(X_train))
    epoch_loss = 0
    
    for i in range(0, len(X_train), BATCH_SIZE):
        batch_idx = indices[i : i + BATCH_SIZE]
        batch_X = X_train[batch_idx]
        batch_y = y_train[batch_idx]
        
        # Forward
        optimizer.zero_grad()
        predictions = model(batch_X)
        
        # Loss
        loss = complex_mse_loss(predictions, batch_y)
        
        # Backward
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    # Validation Step (every 5 epochs)
    if (epoch + 1) % 5 == 0:
        model.eval()
        with torch.no_grad():
            test_preds = model(X_test)
            val_loss = complex_mse_loss(test_preds, y_test)
            avg_fid = calculate_batch_fidelity(test_preds, y_test)
            
        print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {val_loss:.6f} | Avg Fidelity: {avg_fid:.4f}")

# --- 4. Save Model ---
os.makedirs("outputs", exist_ok=True)
save_path = "outputs/model_weights.pth"
torch.save(model.state_dict(), save_path)
print(f"\n✅ Training Complete! Model saved to {save_path}")

Loading dataset...
Data Loaded. Training on 8000 samples, Testing on 2000.

Starting Training for 50 epochs...
Epoch 5/50 | Loss: 0.001052 | Avg Fidelity: 0.7924
Epoch 10/50 | Loss: 0.000877 | Avg Fidelity: 0.7940
Epoch 15/50 | Loss: 0.000813 | Avg Fidelity: 0.7949
Epoch 20/50 | Loss: 0.000834 | Avg Fidelity: 0.7959
Epoch 25/50 | Loss: 0.001048 | Avg Fidelity: 0.7948
Epoch 30/50 | Loss: 0.000821 | Avg Fidelity: 0.7928
Epoch 35/50 | Loss: 0.000760 | Avg Fidelity: 0.7950
Epoch 40/50 | Loss: 0.000850 | Avg Fidelity: 0.7947
Epoch 45/50 | Loss: 0.000831 | Avg Fidelity: 0.7961
Epoch 50/50 | Loss: 0.000759 | Avg Fidelity: 0.7953

✅ Training Complete! Model saved to outputs/model_weights.pth


In [4]:
import torch
import numpy as np
import time

# --- 1. Setup & Load Model ---
# Detect device (CPU is fine for evaluation)
device = torch.device("cpu")

# Load Data
print("Loading test data...")
data = np.load("qst_data_ml/dataset.npz")
X_test = torch.tensor(data['X'][8000:], dtype=torch.float32).to(device) # Last 2000 samples
y_test = torch.tensor(data['y'][8000:], dtype=torch.complex64).to(device)

# Load Model
model = QuantumStateReconstructor().to(device)
model.load_state_dict(torch.load("outputs/model_weights.pth"))
model.eval()

print("Model loaded. Starting evaluation...")

# --- 2. Calculate Metrics (Fidelity & Trace Distance) ---
def compute_metrics(pred_rho, true_rho):
    """
    Computes Fidelity and Trace Distance for a batch.
    Uses PyTorch linear algebra.
    """
    # 1. Trace Distance: T = 0.5 * Tr|rho - sigma| = 0.5 * sum(abs(eigenvalues(diff)))
    diff = pred_rho - true_rho
    # Eigenvalues of the difference matrix
    eigvals = torch.linalg.eigvalsh(diff) 
    trace_dist = 0.5 * torch.sum(torch.abs(eigvals), dim=1)
    
    # 2. Fidelity: F = (Tr(sqrt(sqrt(rho) sigma sqrt(rho))))^2
    # For mixed states, this is hard to vectorize perfectly in vanilla PyTorch.
    # We will use the standard approximation for validation: Tr(rho * sigma) 
    # (Exact for pure states, lower bound for mixed).
    # IF you need exact mixed state fidelity, we loop and use scipy/qutip (slower).
    
    # Let's stick to the overlap metric we used in training for consistency, 
    # but strictly it is 'State Overlap'.
    overlap_matrix = torch.matmul(pred_rho, true_rho)
    overlap = torch.diagonal(overlap_matrix, dim1=1, dim2=2).sum(dim=1).real
    
    # Clamp to [0,1] just in case
    fidelity = torch.clamp(overlap, 0.0, 1.0)
    
    return fidelity, trace_dist

with torch.no_grad():
    # Run Inference on all test data
    start_time = time.time()
    predictions = model(X_test)
    end_time = time.time()
    
    # Calculate Latency
    total_time = end_time - start_time
    latency_per_sample = (total_time / len(X_test)) * 1000 # in ms
    
    # Calculate Physics Metrics
    fidelities, trace_dists = compute_metrics(predictions, y_test)
    
    mean_fidelity = fidelities.mean().item()
    mean_trace = trace_dists.mean().item()

# --- 3. Final Report ---
print("\n" + "="*40)
print("   FINAL PERFORMANCE REPORT (Track 2)")
print("="*40)
print(f"1. Mean Fidelity:       {mean_fidelity:.4f}")
print(f"2. Mean Trace Distance: {mean_trace:.4f}")
print(f"3. Inference Latency:   {latency_per_sample:.4f} ms/sample")
print("="*40)

# Save these numbers for your README!

Loading test data...
Model loaded. Starting evaluation...

   FINAL PERFORMANCE REPORT (Track 2)
1. Mean Fidelity:       0.7953
2. Mean Trace Distance: 0.0179
3. Inference Latency:   0.0211 ms/sample
